<a href="https://colab.research.google.com/github/Shoma1112/Shoma1112/blob/main/%E3%80%90%E7%AC%AC16%E5%9B%9E_Beginner%E9%99%90%E5%AE%9A%E3%82%B3%E3%83%B3%E3%83%9A%E3%80%91%E8%87%AA%E5%8B%95%E8%BB%8A%E3%81%AE%E8%B5%B0%E8%A1%8C%E8%B7%9D%E9%9B%A2%E4%BA%88%E6%B8%AC_ver2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

【第16回_Beginner限定コンペ】自動車の走行距離予測

In [ ]:
import numpy as np
import pandas as pd
import pandas_profiling
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb

from sklearn.model_selection import train_test_split

import statsmodels.formula.api as smf

In [ ]:
data = pd.read_csv("sample_data/train.csv")
test = pd.read_csv("sample_data/test.csv")

In [ ]:
#train.profile_report()

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            1200 non-null   int64  
 1   cylinders     1200 non-null   float64
 2   displacement  1200 non-null   float64
 3   horsepower    1200 non-null   object 
 4   weight        1200 non-null   float64
 5   acceleration  1200 non-null   float64
 6   model year    1200 non-null   float64
 7   origin        1200 non-null   float64
 8   car name      1200 non-null   object 
 9   mpg           1200 non-null   float64
dtypes: float64(7), int64(1), object(2)
memory usage: 93.9+ KB


In [ ]:
#label化
from sklearn.preprocessing import LabelEncoder

train = pd.concat([data, test], sort=False)

le = LabelEncoder()
encoded = le.fit_transform(train['car name'].values)
train['car name'] = encoded
train['car name'] = train["car name"].astype("category")

In [ ]:
data = train.iloc[:len(data)]
test = train.iloc[len(data):]

In [ ]:
#変数選択
threshold = 0.8

X = data.iloc[:, 1:9]
corr_matrix = X.corr().abs()
corr_matrix

,cylinders,displacement,weight,acceleration,model year,origin
cylinders,1.000000,0.757172,0.319396,0.431085,0.370580,0.221007
displacement,0.757172,1.000000,0.393332,0.341080,0.457503,0.286420
weight,0.319396,0.393332,1.000000,0.048476,0.264717,0.041376
acceleration,0.431085,0.341080,0.048476,1.000000,0.190724,0.056397
model year,0.370580,0.457503,0.264717,0.190724,1.000000,0.050123
origin,0.221007,0.286420,0.041376,0.056397,0.050123,1.000000


In [ ]:
test.head()

,id,cylinders,displacement,horsepower,weight,acceleration,model year,origin,car name,mpg
0,1200,4.002,97.502470,71,2722.792216,15.419528,76.434109,0.998000,11,NaN
1,1201,4.002,132.430995,150,1794.227186,20.032104,76.774002,0.998000,58,NaN
2,1202,4.002,100.040582,150,2103.863515,13.415063,70.997969,0.998000,21,NaN
3,1203,6.002,400.452581,85,2722.502681,17.665557,71.431953,0.998000,9,NaN
4,1204,4.002,143.165887,90,3277.078489,15.517748,80.722361,0.998001,44,NaN


In [ ]:
#欠損値"?"を埋める

data = data.replace("?", np.NaN)

data["horsepower"] = data["horsepower"].astype('float64')
lm = smf.ols(formula='horsepower ~ cylinders', data=data).fit()
data["horsepower"] = data["horsepower"].fillna(lm.predict(data["cylinders"]))

data.isnull().sum()

id              0
cylinders       0
displacement    0
horsepower      0
weight          0
acceleration    0
model year      0
origin          0
car name        0
mpg             0
dtype: int64

In [ ]:
X = data.drop(columns=["mpg", "id"])
y = data["mpg"]

In [ ]:
test_id = test["id"]
#test = test.drop(columns=["id", "mpg"])
test = test.drop(columns=["id"])

test = test.replace("?", np.NaN)
test["horsepower"] = test["horsepower"].astype('float64')
lm = smf.ols(formula='horsepower ~ cylinders', data=test).fit()
test["horsepower"] = test["horsepower"].fillna(lm.predict(data["cylinders"]))
test.isnull().sum()

cylinders         0
displacement      0
horsepower        0
weight            0
acceleration      0
model year        0
origin            0
car name          0
mpg             800
dtype: int64

In [ ]:
#train, testに分ける
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=0)

In [ ]:
#lightgbm model
lgb_train = lgb.Dataset(X_train, y_train, categorical_feature=['car name'])
lgb_eval =  lgb.Dataset(X_valid, y_valid, categorical_feature=['car name'])

params = {'objective':'rmse', 'max_bin':300, 'learning_rate':0.05, 'num_leaves':200}

model = lgb.train(params, lgb_train, valid_sets=[lgb_train, lgb_eval],
                  verbose_eval=10,
                  num_boost_round=1000,
                  early_stopping_rounds=200)

y_pred_lgb = model.predict(test, num_iteration=model.best_iteration)

/usr/local/lib/python3.7/dist-packages/lightgbm/basic.py:1205: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/usr/local/lib/python3.7/dist-packages/lightgbm/basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 200 rounds.
[10]	training's rmse: 5.01722	valid_1's rmse: 5.25848
[20]	training's rmse: 3.62237	valid_1's rmse: 4.02773
[30]	training's rmse: 2.85516	valid_1's rmse: 3.44118
[40]	training's rmse: 2.4084	valid_1's rmse: 3.15988
[50]	training's rmse: 2.13866	valid_1's rmse: 3.043
[60]	training's rmse: 1.95521	valid_1's rmse: 2.98111
[70]	training's rmse: 1.81142	valid_1's rmse: 2.94281
[80]	training's rmse: 1.69594	valid_1's rmse: 2.91376
[90]	training's rmse: 1.59663	valid_1's rmse: 2.91333
[100]	training's rmse: 1.50502	valid_1's rmse: 2.91205
[110]	training's rmse: 1.42008	valid_1's rmse: 2.90294
[120]	training's rmse: 1.3512	valid_1's rmse: 2.89497
[130]	training's rmse: 1.28378	valid_1's rmse: 2.88243
[140]	training's rmse: 1.22527	valid_1's rmse: 2.88173
[150]	training's rmse: 1.17683	valid_1's rmse: 2.88442
[160]	training's rmse: 1.12825	valid_1's rmse: 2.88229
[170]	training's rmse: 1.08051	valid_1's rmse: 2.88409
[180]	training'

In [ ]:
sub_pred = pd.DataFrame(data=y_pred_lgb)

sub = pd.concat([test_id, sub_pred], axis=1, sort=False)
sub.to_csv("submit.csv", header=False, index=False)
sub.head()

,id,0
0,1200,31.708998
1,1201,25.330108
2,1202,29.318714
3,1203,21.043939
4,1204,31.217967
